# Arithmetic-RNN

In [4]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
# tf.reset_default_graph()
session = tf.InteractiveSession()
import utils
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x10bed4d90>> ignored


In [2]:
X, y, index_to_word, equations, answers, max_length = utils.load_arithmetic_data()
X_train, y_train, X_test, y_test = utils.split_data(X, y)
vocab_size = len(index_to_word)
n_classes = y.shape[1]

s_i = 50
print("Equation:", equations[s_i])
print("Answer:", answers[s_i])

1324


KeyError: '0'

In [42]:
data_placeholder = tf.placeholder(tf.float32, shape=(None, max_length, vocab_size), name='data_placeholder')
labels_placeholder = tf.placeholder(tf.float32, shape=(None, max_length, vocab_size), name='labels_placeholder')
keep_prob_placeholder = tf.placeholder(tf.float32, name='keep_prob_placeholder')

In [ ]:
def rnn_decoder(decoder_inputs, initial_state, cell, loop_function=None,
                scope=None):
  """RNN decoder for the sequence-to-sequence model.
  Args:
    decoder_inputs: A list of 2D Tensors [batch_size x input_size].
    initial_state: 2D Tensor with shape [batch_size x cell.state_size].
    cell: rnn_cell.RNNCell defining the cell function and size.
    loop_function: If not None, this function will be applied to the i-th output
      in order to generate the i+1-st input, and decoder_inputs will be ignored,
      except for the first element ("GO" symbol). This can be used for decoding,
      but also for training to emulate http://arxiv.org/abs/1506.03099.
      Signature -- loop_function(prev, i) = next
        * prev is a 2D Tensor of shape [batch_size x output_size],
        * i is an integer, the step number (when advanced control is needed),
        * next is a 2D Tensor of shape [batch_size x input_size].
    scope: VariableScope for the created subgraph; defaults to "rnn_decoder".
  Returns:
    A tuple of the form (outputs, state), where:
      outputs: A list of the same length as decoder_inputs of 2D Tensors with
        shape [batch_size x output_size] containing generated outputs.
      state: The state of each cell at the final time-step.
        It is a 2D Tensor of shape [batch_size x cell.state_size].
        (Note that in some cases, like basic RNN cell or GRU cell, outputs and
         states can be the same. They are different for LSTM cells though.)
  """
  with variable_scope.variable_scope(scope or "rnn_decoder"):
    state = initial_state
    outputs = []
    prev = None
    for i, inp in enumerate(decoder_inputs):
      if loop_function is not None and prev is not None:
        with variable_scope.variable_scope("loop_function", reuse=True):
          inp = loop_function(prev, i)
      if i > 0:
        variable_scope.get_variable_scope().reuse_variables()
      output, state = cell(inp, state)
      outputs.append(output)
      if loop_function is not None:
        prev = output
  return outputs, state

def linear(input_, output_size, layer_scope, stddev=0.02, bias_start=0.0):
    shape = input_.get_shape().as_list()

    with tf.variable_scope(layer_scope, reuse=True):
        matrix = tf.get_variable("Matrix", [shape[1], output_size], tf.float32,
                                 tf.random_normal_initializer(stddev=stddev))
        bias = tf.get_variable("bias", [output_size],
            initializer=tf.constant_initializer(bias_start))
        return tf.matmul(input_, matrix) + bias

In [46]:
# Define Computation Graph
n_rnn_layers = 3
n_fc_layers = 2
n_rnn_nodes = 256
n_fc_nodes = 128
batch_size = 64

with tf.name_scope("encoder_layers") as scope:
    # Create LSTM Cell
    cell = tf.nn.rnn_cell.LSTMCell(n_rnn_nodes, state_is_tuple=False)
    cell = tf.nn.rnn_cell.DropoutWrapper(
        cell, output_keep_prob=keep_prob_placeholder)
    stacked_cells = tf.nn.rnn_cell.MultiRNNCell([cell] * n_rnn_layers, state_is_tuple=False)
    output, encoding = tf.nn.dynamic_rnn(stacked_cells, data_placeholder, dtype=tf.float32)

with tf.name_scope("decoder_layers") as scope:
    # Connect RNN Embedding output to decoder
    initial_state = tf.zeros([batch_size, n_rnn_nodes], tf.float32)
    encoder_cell = tf.nn.rnn_cell.LSTMCell(n_rnn_nodes, state_is_tuple=False)
    encoder_list = [encoder for i in range(max_length)]
    outputs, state = tf.nn.seq2seq.rnn_decoder(encoder, initial_state, encoder_cell)
    
    fc0_output = [tf.nn.relu(linear(data_placeholder, n_hidden_units_h0, 'hidden0')) for output in outputs]
    output_shape = [-1, self.n_features, self.sequence_length, 2]

    predictions = tf.reshape(fc2_output, output_shape)
    fc_final = linear(fc_prev, n_classes, 'fc{}'.format(n_fc_layers-1))

logits = tf.nn.softmax(fc_final)

TypeError: name_scope() got an unexpected keyword argument 'reuse'

In [ ]:
# Define Loss Function + Optimizer
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, labels_placeholder))

weights = [tf.constant(1.0, shape=[2]) for i in range(3)]
loss = sequence_loss(logits, label_placeholder, weights)

optimizer = tf.train.GradientDescentOptimizer(0.0002).minimize(loss)
prediction = tf.nn.softmax(logits)
prediction_is_correct = tf.equal(
    tf.argmax(logits, 1), tf.argmax(labels_placeholder, 1))
accuracy = tf.reduce_mean(tf.cast(prediction_is_correct, tf.float32))

In [ ]:
# Train loop

num_steps = 1000
batch_size = 32
keep_prob_rate = 0.75

tf.initialize_all_variables().run()

for step in xrange(num_steps):
    offset = (step * batch_size) % (X_train.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = X_train[offset:(offset + batch_size), :, :]
    batch_labels = y_train[offset:(offset + batch_size), :]
    # We built our networking using placeholders. It's like we've made reservations for a party of 6.
    # So use feed_dict to fill what we reserved. And we can't show up with 9 people. 

    feed_dict_train = {data_placeholder: batch_data, labels_placeholder : batch_labels, keep_prob_placeholder: keep_prob_rate}
    # Run the optimizer, get the loss, get the predictions.
    # We can run multiple things at once and get their outputs
    _, loss_value_train, predictions_value_train, accuracy_value_train = session.run(
      [optimizer, loss, prediction, accuracy], feed_dict=feed_dict_train)
    if (step % 2 == 0):
        print "Minibatch train loss at step", step, ":", loss_value_train
        print "Minibatch train accuracy: %.3f%%" % accuracy_value_train
        feed_dict_test = {data_placeholder: X_test, labels_placeholder: y_test, keep_prob_placeholder: 1.0}
        loss_value_test, predictions_value_test, accuracy_value_test = session.run(
            [loss, prediction, accuracy], feed_dict=feed_dict_test)
        print "Test loss: %.3f" % loss_value_test
        print "Test accuracy: %.3f%%" % accuracy_value_test


Minibatch train loss at step 0 : 1.09861
Minibatch train accuracy: 0.469%
Test loss: 1.099
Test accuracy: 0.468%
Minibatch train loss at step 2 : 1.0986
Minibatch train accuracy: 0.438%
Test loss: 1.099
Test accuracy: 0.431%
Minibatch train loss at step 4 : 1.0986
Minibatch train accuracy: 0.469%
Test loss: 1.099
Test accuracy: 0.483%
Minibatch train loss at step 6 : 1.0986
Minibatch train accuracy: 0.469%
Test loss: 1.099
Test accuracy: 0.565%
Minibatch train loss at step 8 : 1.0986
Minibatch train accuracy: 0.406%
Test loss: 1.099
Test accuracy: 0.567%
Minibatch train loss at step 10 : 1.09859
Minibatch train accuracy: 0.500%
Test loss: 1.099
Test accuracy: 0.557%
Minibatch train loss at step 12 : 1.0986
Minibatch train accuracy: 0.438%
Test loss: 1.099
Test accuracy: 0.542%
Minibatch train loss at step 14 : 1.09862
Minibatch train accuracy: 0.188%
Test loss: 1.099
Test accuracy: 0.506%
Minibatch train loss at step